In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
experiment = "kaggle_hashcode"

In [3]:
filepath = os.path.join("input_data", experiment+".txt")
f = open(filepath)

We need the incoming streets of each intersection

We can mark the streets as used when they are part of a schedule

We could count how many times they are part of a schedule and omit them, if they only play a very small role. 

{'intersection_id': {'incoming_street_1' : counter, 
                     'incoming_street_2' : counter} }

routes: street-1  street-2  street-3  street-4
                I1        I2        I3        

we need the intersection at the end of street-1 -> I1

use street_connector mapping, key "E"


In [4]:
first_line = f.readline()
vals = first_line.split(" ")
D = int(vals[0])
I = int(vals[1])
S = int(vals[2])
V = int(vals[3])
F = int(vals[4])
print("D={}, I={}, S={}, V={}, F={}".format(D,I,S,V,F))

D=7071, I=8000, S=63968, V=1000, F=1000


In [5]:
# Create mapping streetname to id {"streetname": id}
mapping_streetnames = {}
inverse_mapping = []
outgoing_streets = {}
incoming_streets = {}
street_connectors = {}
street_durations = {}

i = 0;
while i < S:
    l = f.readline()
    vals = l.split(" ")
    B = int(vals[0])     # intersection beginning of the street
    E = int(vals[1])     # intersection end of the street
    name = vals[2]       # street name
    L = int(vals [3]) 
    mapping_streetnames[name] = i
    inverse_mapping.append(name)
    
    try:
        streets = outgoing_streets[B]
        streets.append(name)
        outgoing_streets[B] = streets
    except:
        # key Error
        outgoing_streets[B] = [name]
    
    try:
        streets = incoming_streets[E]
        streets.append(name)
        incoming_streets[E] = streets
    except:
        # key Error
        incoming_streets[E] = [name]
    
    street_connectors[name] = {"B": B, "E": E}
    street_durations[name] = L
    i += 1


In [6]:
# {'intersection_id':{'incoming_street_1' : counter, 
#                     'incoming_street_2' : counter
#                    } 
# }

I_map = {}

def count_passing_car(intersection_id, incoming_street):
    
    if intersection_id in I_map.keys():
        incoming_streets_dict = I_map[intersection_id]
        
        if incoming_street in incoming_streets_dict:
            incoming_streets_dict[incoming_street] += 1
        
        else:
            incoming_streets_dict[incoming_street] = 1
        
        I_map[intersection_id] = incoming_streets_dict
    
    else:
        I_map[intersection_id] = {incoming_street : 1}
    
        

In [7]:
for v in range(V): 
    l = f.readline()
    vals = l.split("\n")[0].split(" ")

    for i in range(1,int(vals[0])): # ignore the intersection at the end of the last street
        
        street_on_path = vals[i]
        next_intersection = street_connectors[street_on_path]['E']
        count_passing_car(next_intersection, street_on_path)
      
        

In [8]:
num_intersections = len(I_map.keys())
print("I: ", I)
print("Number of intersections: {}".format(num_intersections))

I:  8000
Number of intersections: 7999


In [12]:
# write output file

# number of intersection for which the schedule is defined
# intersection id
# number of incoming streets for which the schedule is defined
# street-name duration

all_intersections = I_map.keys()

with open(os.path.join("generated_schedule", experiment+"_schedule_omit_incoming_streets.out"), "w") as f:
    
    f.write("{}\n".format(len(all_intersections)))
    
    for intersection_id in all_intersections:
        f.write("{}\n".format(intersection_id))
        
        streets = I_map[intersection_id].keys()
        
        f.write("{}\n".format(len(streets)))
        
        for street in streets:
            f.write("{} {}\n".format(street, 1)) #fixed value of 1
        
